# Params for Simple_RNN

$params = cell\times input\_size + cell^2$

$cell^2$ means cells are fullly connected to themselves   meaning?


In [0]:
%tensorflow_version 1.x

In [2]:
# colab.research.google.com
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import numpy as np

'''
# labs.cognitiveclass.ai
import keras
from keras.layers import *
import numpy as np
'''

'\n# labs.cognitiveclass.ai\nimport keras\nfrom keras.layers import *\nimport numpy as np\n'

# Prepare Training Data

In [3]:
BIT = 4

def get_bit(n, bit):
    a = []
    for i in range(bit):
        a.append(n & 1)
        n >>= 1
    return a
  
for i in range(4):
    print(get_bit(i, BIT))


[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[1, 1, 0, 0]


In [0]:
time_steps = BIT

x = []
y = []
for i in range(2**time_steps):
    for j in range(2**time_steps):
        i_b = get_bit(i, time_steps)
        j_b = get_bit(j, time_steps)
        x.append([[i_b[i], j_b[i]] for i in range(time_steps)])
        y.append([get_bit(i+j, time_steps)[-1]])

In [5]:
x = np.array(x)
x.shape

(256, 4, 2)

In [6]:
y = np.array(y)
y.shape

(256, 1)

# Train

In [7]:
cell = 8
input_size = 2
time_steps = BIT    

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        # LSTM(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        Dense(1, activation='relu', use_bias=False)    
])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 8)                 80        
_________________________________________________________________
dense (Dense)                (None, 1)                 8         
Total params: 88
Trainable params: 88
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', #tf.keras.optimizers.RMSprop(learning_rate=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(x, y, epochs=2000, verbose=0)    
print(history.history.keys())
#model.fit(x, y, epochs=2000)    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
dict_keys(['loss', 'acc'])


In [10]:
print(history.history['loss'][-1])
print(history.history['acc'][-1])

0.001049817178682133
1.0


# Test

In [11]:
a = model.predict_classes(x)
all(a == y)

True

In [0]:
def add(a, b):
  a = get_bit(a, BIT)
  b = get_bit(b, BIT)
  x = [[a[i], b[i]] for i in range(BIT)]
  x = [[x]]
  return model.predict_classes(x)[0][0]

In [0]:
assert add(9, 0) == 1
assert add(14, 2) == 0 
assert add(15, 7) == 0